In [1]:
from ortools.graph import pywrapgraph
import pandas as pd
import numpy as np

from functions.utils import *

## Transportation Problem 

Goal is to allocate the clients (25) which must be supplied by the different Providers (5) in an optimal way so that all demand is satisfied using the nodes in the following graph. 

<img src="img/LargerProblem.jpeg">

In [2]:
# Reference:
# https://developers.google.com/optimization/flow/mincostflow
# https://towardsdatascience.com/operations-research-in-r-transportation-problem-1df59961b2ad
def solve_transportation_problem(start_nodes, end_nodes, capacities, unit_costs, supplies ):
    """MinCostFlow problem solver using Google OR Tools. 
    
    Args:
        start_nodes (list): start nodes at every edge.
        end_nodes (list): end nodes at every edge.
        capacities (list): array with capacities for every edge.
        unit_cost (list): array with cost for every edge
        supplies (list): array with demand/supply at every node.

    """

    # Instantiate a SimpleMinCostFlow solver.
    min_cost_flow = pywrapgraph.SimpleMinCostFlow()

    # Add each arc.
    for arc in zip(start_nodes, end_nodes, capacities, unit_costs):
        min_cost_flow.AddArcWithCapacityAndUnitCost(arc[0], arc[1], arc[2],
                                                    arc[3])

    # Add node supply.
    for count, supply in enumerate(supplies):
        min_cost_flow.SetNodeSupply(count, supply)

    # Find the min cost flow.
    status = min_cost_flow.Solve()

    if status != min_cost_flow.OPTIMAL:
        print('There was an issue with the min cost flow input.')
        print(f'Status: {status}')
        if status == min_cost_flow.INFEASIBLE:
            print("The problem is infeasible")
        if status == min_cost_flow.UNBALANCED:
            print("The problem is unbalanced")


    print('Minimum cost: ', min_cost_flow.OptimalCost())
    print('')
    print(' Arc   Flow / Capacity  Cost')
    for i in range(min_cost_flow.NumArcs()):
        cost = min_cost_flow.Flow(i) * min_cost_flow.UnitCost(i)
        print('%1s -> %1s    %3s   / %3s   %3s' %
              (min_cost_flow.Tail(i), min_cost_flow.Head(i),
               min_cost_flow.Flow(i), min_cost_flow.Capacity(i), cost))
    

## 1. Read in Data

In [9]:
# Read in Data
df_cap = pd.read_excel('docs/io_v2.xlsx', 
            sheet_name = 'Capacities', 
            usecols = [0,1,2], 
             dtype = {
                        'NodeIndex': str, 
                        'NodeName': str, 
                        'Capacity': float}).dropna()

# Load Nodes connections
df_conn = pd.read_excel('docs/io_v2.xlsx', 
                    sheet_name = 'Connections', usecols = [0,1,2,3], 
                    dtype = {
                        'StartNode': str, 
                        'EndNode': str, 
                        'Distance': float,
                        'Cost': float}).dropna()

## 2. Preprocess Data

In [ ]:
# Provider 6 is only considered in the Shortest Path Problem
# In order to generate other variations of the problem.
# Provider names can be added so that it becomes, 
# an imbalanced supply or imbalanced demand problem, 
# by removing Providers in the network.  
# ex. ['Prov 5','Prov 6']
prov = ['Prov 5','Prov 6']
df_cap, df_conn = eliminate_provider(df_cap, df_conn, prov)

In [10]:
# Calculate the total supply and demand in the netwrok
cap_prov = df_cap.loc[df_cap['NodeName'].apply(lambda x: 'Prov' in x)]['Capacity'].sum()
cap_client = df_cap.loc[df_cap['NodeName'].apply(lambda x: 'Cliente' in x)]['Capacity'].sum()

# Check for imbalance problem and adjust it
df_cap, df_conn = adjust_umbalance_problem(df_cap, df_conn, cap_prov, cap_client )

# Reindex Nodes
df_cap['NodeIndex'] = df_cap.index

# Drop missing values
df_cap.dropna(inplace = True)

# Adjust Capacity sign
# This step is needed since Google OR Tools considers demands as negative supply values
df_cap['sign'] = df_cap['NodeName'].apply(lambda x: -1 if 'Cliente' in x else 1)
df_cap['Capacity'] = df_cap['Capacity'] * df_cap['sign']


# Trim Start and EndNode
# This step is needed to avoid mismatch in the node index conversion
cols = ['StartNode', 'EndNode']
df_conn = strip_col(df_conn, cols)

# Trim Node name
cols = ['NodeName']
df_cap = strip_col(df_cap, cols)

# Map nodes names to their corresponding indeces
node_mapper = dict(zip(df_cap['NodeName'], df_cap['NodeIndex'].astype(int)))
df_conn['StartNodeIndex'] = df_conn['StartNode'].apply(lambda x: int(node_mapper.get(x)) )
df_conn['EndNodeIndex'] = df_conn['EndNode'].apply(lambda x: int(node_mapper.get(x)) )
df_conn['Total_Cost'] = (df_conn['Distance'] * df_conn['Cost'] * 10).astype(int)

 Problem is imbalanced the demand is higher than the supply
 Adding artifical Provider that delivers missing supply


## 3. Generate Input Vectors

In [11]:
# Create Input Vectors
c = 10**5

start_nodes = df_conn['StartNodeIndex'].to_list()
end_nodes = df_conn['EndNodeIndex'].to_list()
capacities = [c] * df_conn.shape[0]
unit_costs = df_conn['Total_Cost'].astype(int).to_list() 

# Define an array of supplies at each node.
supplies = df_cap['Capacity'].astype(int).to_list()

In [12]:
node_mapper

{'Prov 1': 0,
 'Prov 2': 1,
 'Prov 3': 2,
 'Prov 4': 3,
 'Cliente 1': 4,
 'Cliente 2': 5,
 'Cliente 3': 6,
 'Cliente 4': 7,
 'Cliente 5': 8,
 'Cliente 6': 9,
 'Cliente 7': 10,
 'Cliente 8': 11,
 'Cliente 9': 12,
 'Cliente 10': 13,
 'Cliente 11': 14,
 'Cliente 12': 15,
 'Cliente 13': 16,
 'Cliente 14': 17,
 'Cliente 15': 18,
 'Cliente 16': 19,
 'Cliente 17': 20,
 'Cliente 18': 21,
 'Cliente 19': 22,
 'Cliente 20': 23,
 'Cliente 21': 24,
 'Cliente 22': 25,
 'Cliente 23': 26,
 'Cliente 24': 27,
 'Cliente 25': 28,
 'Prov Add': 29}

## 4. Solve Transportation Problem

In [13]:
solve_transportation_problem(start_nodes, end_nodes, capacities, unit_costs, supplies)

Minimum cost:  47962

 Arc   Flow / Capacity  Cost
0 -> 4      0   / 100000     0
0 -> 5      0   / 100000     0
0 -> 6      0   / 100000     0
0 -> 7     44   / 100000   572
0 -> 8     59   / 100000   1711
0 -> 9      0   / 100000     0
0 -> 10      0   / 100000     0
0 -> 11      0   / 100000     0
0 -> 12      0   / 100000     0
0 -> 13     60   / 100000   1440
0 -> 14      0   / 100000     0
0 -> 15      0   / 100000     0
0 -> 16     86   / 100000   3440
0 -> 17      0   / 100000     0
0 -> 18      0   / 100000     0
0 -> 19      0   / 100000     0
0 -> 20      0   / 100000     0
0 -> 21     38   / 100000   1102
0 -> 22      0   / 100000     0
0 -> 23      0   / 100000     0
0 -> 24      0   / 100000     0
0 -> 25      0   / 100000     0
0 -> 26    143   / 100000   4433
0 -> 27      0   / 100000     0
0 -> 28      0   / 100000     0
1 -> 4      0   / 100000     0
1 -> 5     33   / 100000   1287
1 -> 6    115   / 100000   2875
1 -> 7      0   / 100000     0
1 -> 8     37   / 100000